In [34]:
import os
import pandas as pd
import path
import subprocess
from subprocess import Popen, PIPE
import json

In [48]:
DIB_dir = "DIB/"
NCGR_dir = "NCGR/"
DIB_files = os.listdir(DIB_dir)
print(DIB_files)
NCGR_files = os.listdir(NCGR_dir)
print(NCGR_files)

['missing_busco_list_MMETSP0045.tsv', 'missing_busco_list_MMETSP0121.tsv', 'missing_busco_list_MMETSP0232.tsv', 'missing_busco_list_MMETSP0169.tsv', 'missing_busco_list_MMETSP0439.tsv', 'missing_busco_list_MMETSP0717.tsv', 'missing_busco_list_MMETSP0932.tsv', 'missing_busco_list_MMETSP0329.tsv']
['missing_busco_list_MMETSP0045.tsv', 'missing_busco_list_MMETSP0121.tsv', 'missing_busco_list_MMETSP0232.tsv', 'missing_busco_list_MMETSP0169.tsv', 'missing_busco_list_MMETSP0439.tsv', 'missing_busco_list_MMETSP0717.tsv', 'missing_busco_list_MMETSP0932.tsv', 'missing_busco_list_MMETSP0329.tsv']


In [53]:
# What is in NCGR not in DIB?
common_dib = []
for missing_file in DIB_files:
    MMETSP = missing_file.split("_")[3]
    print(MMETSP[:-4])
    df_dib = pd.read_csv(DIB_dir+missing_file,skiprows=3)
    df_dib.columns = ['orthodb']
    print("Missing in DIB:",len(df_dib))
    for missing_file2 in NCGR_files:
        sample = missing_file2.split("_")[3]
        if MMETSP == sample:
            df_ncgr = pd.read_csv(NCGR_dir+missing_file2,skiprows=3)
            df_ncgr.columns = ['orthodb']
            print("Missing in NCGR:",len(df_ncgr))
            unique_dib = df_dib[~df_dib.orthodb.isin(df_ncgr.orthodb)].dropna()
            print("Missing only in DIB (should be present in NCGR):",len(unique_dib))
            dib_list = unique_dib['orthodb'].values.tolist()
            for i in dib_list:
                common_dib.append(i)

MMETSP0045
Missing in DIB: 142
Missing in NCGR: 51
Missing only in DIB (should be present in NCGR): 91
MMETSP0121
Missing in DIB: 130
Missing in NCGR: 77
Missing only in DIB (should be present in NCGR): 53
MMETSP0232
Missing in DIB: 294
Missing in NCGR: 30
Missing only in DIB (should be present in NCGR): 264
MMETSP0169
Missing in DIB: 215
Missing in NCGR: 49
Missing only in DIB (should be present in NCGR): 171
MMETSP0439
Missing in DIB: 282
Missing in NCGR: 39
Missing only in DIB (should be present in NCGR): 243
MMETSP0717
Missing in DIB: 141
Missing in NCGR: 51
Missing only in DIB (should be present in NCGR): 91
MMETSP0932
Missing in DIB: 265
Missing in NCGR: 39
Missing only in DIB (should be present in NCGR): 226
MMETSP0329
Missing in DIB: 280
Missing in NCGR: 37
Missing only in DIB (should be present in NCGR): 243


In [10]:
print(len(common_dib))

1382


In [11]:
occurences = {}
for i in common_dib:
    if i in occurences:
        occurences[i]+=1
    else:
        occurences[i] = 1
print('Unique orthodb:',len(occurences))
sort = [(k, occurences[k]) for k in sorted(occurences, key=occurences.get, reverse=True)]

Unique orthodb: 302


In [54]:
more_than_one = []
for s in sort:
    if int(s[1]) >= 8:
        more_than_one.append(s[0])

In [55]:
print(len(more_than_one))

5


In [56]:
print(more_than_one)

['EOG0937029K', 'EOG09370F1Q', 'EOG09370KJ3', 'EOG09370OHP', 'EOG09370S0F']


In [57]:
# busco API
# https://www.orthodb.org/?page=api
# format: http://www.orthodb.org/CMD?ARG1="value"&ARG2="value&

In [58]:
for i in more_than_one:
    command_parts = ['wget ','http://www.orthodb.org','/group?','id=',i,' -O ',i,'.orthodb']
    command = ''.join(command_parts)
    #print(command)
    #s = subprocess.Popen(command, shell=True)
    #s.wait()
    eog_filename = i+'.orthodb'
    with open(eog_filename) as r:
        lines = r.readlines()
        j = json.loads(lines[0])
        data = j["data"]
        name = data["name"]
        print(name)

Vps54-like
apoptosis antagonizing transcription factor 
Exosome complex RNA-binding protein 1/RRP40/RRP4
tRNA (1-methyladenosine) methyltransferase catalytic subunit Gcd14
Ribosome biogenesis protein
